# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt','wordnet','averaged_perceptron_tagger'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# load data from database
engine = create_engine('sqlite:///Cleandata.db')
df = pd.read_sql("SELECT * FROM Cleandata", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
Y = df.iloc[:,4:40]
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    #text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # Punctuation Removal
    
    tokens=word_tokenize(text)   # tokenization
    
    lemmatizer=WordNetLemmatizer()  
    
    clean_tokens =[]
    
    for tok in tokens:
        
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        clean_tokens.append(clean_tok)
        
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect',  CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [9]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
Y_pred=pipeline.predict(X_test)

In [10]:
Y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [12]:
colname=list(Y_test.columns)
print(colname)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [13]:
# modularize the classification_report
def classreport(Y_test, Y_pred):
    """
    modularize the classification_report
    
    """
    colname=list(Y_test.columns)
    
    Y_pred2=pd.DataFrame(Y_pred, index=Y_pred[:,0])
    
    target_names=['0','1']

    a=classification_report(Y_test, Y_pred2, target_names=target_names)
    
    avg_total_f1=float(a[-17:-9])
    
    return avg_total_f1

In [25]:
def evaluate_model(model, X_test, Y_test, category_names):
    
    """
    
    output f1 score, precision and recall for the test set
    for each category
    
    """
    Y_pred=pipeline.predict(X_test)
    
    Y_pred2=pd.DataFrame(Y_pred, index=Y_pred[:,0], columns=category_names)
    
    target_names=['0','1']
    
    for a in category_names:
        print("Evaluation for", a)
        print(classification_report(Y_test[a], Y_pred2[a], target_names=target_names))
        
    return None    

In [26]:
evaluate_model(pipeline, X_test, Y_test, colname)

Evaluation for related
             precision    recall  f1-score   support

          0       0.59      0.34      0.43      1518
          1       0.82      0.93      0.87      5027

avg / total       0.77      0.79      0.77      6545

Evaluation for request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5442
          1       0.79      0.39      0.53      1103

avg / total       0.87      0.88      0.86      6545

Evaluation for offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6512
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6545

Evaluation for aid_related
             precision    recall  f1-score   support

          0       0.72      0.86      0.79      3849
          1       0.73      0.52      0.61      2696

avg / total       0.72      0.72      0.71      6545

Evaluation for medical_help
             preci

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [10]:
def evaluate_model2(model, X_test, Y_test, category_names):
    
    """
    
    output f1 score, precision and recall for the test set
    for each category
    
    """
    Y_pred=pipeline.predict(X_test)
    
    Y_pred2=pd.DataFrame(Y_pred, index=Y_pred[:,0], columns=category_names)
    
    target_names=['0','1']
    
    print(classification_report(Y_test, Y_pred2, target_names=target_names))
        
    return None    

In [13]:
evaluate_model2(pipeline, X_test, Y_test, colname)

             precision    recall  f1-score   support

          0       0.83      0.93      0.88      5079
          1       0.85      0.37      0.52      1139

avg / total       0.77      0.45      0.51     20890



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 36, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:
# def evaluate_model(model, X_test, Y_test, category_names):
colname=list(Y_test.columns)

In [17]:
Y_pred2=pd.DataFrame(Y_pred, index=Y_pred[:,0], columns=colname)

In [23]:
print(classification_report(Y_test['related'], Y_pred2['related'], target_names=target_names))

             precision    recall  f1-score   support

          0       0.59      0.34      0.43      1518
          1       0.82      0.93      0.87      5027

avg / total       0.77      0.79      0.77      6545



In [19]:
target_names=['0','1']

1    1
1    1
1    1
1    1
1    1
1    1
0    0
0    0
1    1
1    1
0    0
1    1
1    1
1    1
1    1
0    0
1    1
0    0
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
0    0
1    1
1    1
0    0
    ..
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
0    0
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
1    1
0    0
1    1
1    1
1    1
1    1
Name: related, Length: 6545, dtype: int64

In [20]:
classification_report(Y_test, Y_pred2, target_names=target_names)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 36, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


'             precision    recall  f1-score   support\n\n          0       0.82      0.93      0.87      5027\n          1       0.79      0.39      0.53      1103\n\navg / total       0.73      0.45      0.50     20751\n'

### different machine learning algorithm

In [14]:
# Cross-validate for pipeline 1
#cross_val_score(pipeline, X, Y, cv=3, scoring='f1')

In [15]:
cv_results = cross_validate(pipeline, X, Y, cv=3)

In [16]:
cv_results['test_score']

array([ 0.25037241,  0.19857912,  0.17522347])

In [17]:
re=cv_results['test_score'].mean()

In [18]:
re

0.20805833327489745

In [21]:
def cvscore(pipeline, X, Y, cv=3):
    """
    return the average test score of cross_validation
    
    """
    cv_results = cross_validate(pipeline, X, Y, cv=cv)
    
    re=cv_results['test_score'].mean()
    
    return re

In [22]:
re1=cvscore(pipeline, X, Y, 3)
re1

0.20912781551774703

In [23]:
re2=cvscore(pipeline, X, Y, 5)
re2

0.21310160427807484

In [116]:
cv_results

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 65.81975722,  56.14768934,  55.53346372]),
 'score_time': array([ 5.26799726,  6.08702493,  6.6382978 ]),
 'test_score': array([ 0.25633093,  0.17990146,  0.17591107]),
 'train_score': array([ 0.79751332,  0.8295422 ,  0.82978114])}

## Mrudula's suggestion

In [27]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [28]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
Y['test'] = Y.sum(axis=1)

In [30]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,test
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,5
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
Y['Total'] = np.where(Y['test'] == 0, 1,0) 

In [33]:
Y.head(100)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,test,Total
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,5,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,8,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,3,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,6,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,6,0


In [34]:
Y.drop('test',axis=1,inplace=True)

In [35]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,Total
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
cv_results3 = cross_validate(pipeline, X, Y, cv=3)

In [37]:
cv_results3

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 72.00828838,  61.74091482,  60.83848166]),
 'score_time': array([ 6.18243814,  6.80645156,  7.09098005]),
 'test_score': array([ 0.15973416,  0.17268248,  0.16857667]),
 'train_score': array([ 0.7697244 ,  0.80639432,  0.81093159])}

In [98]:
cv_results2 = cross_validate(pipeline, X, Y, cv=3, scoring=('f1'))

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.

In [108]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [107]:
Y.related.value_counts()

1    20064
0     6116
Name: related, dtype: int64

In [109]:
for i in Y.columns:
    print(Y[i].value_counts())

1    20064
0     6116
Name: related, dtype: int64
0    21716
1     4464
Name: request, dtype: int64
0    26062
1      118
Name: offer, dtype: int64
0    15339
1    10841
Name: aid_related, dtype: int64
0    24099
1     2081
Name: medical_help, dtype: int64
0    24869
1     1311
Name: medical_products, dtype: int64
0    25456
1      724
Name: search_and_rescue, dtype: int64
0    25709
1      471
Name: security, dtype: int64
0    25321
1      859
Name: military, dtype: int64
0    26180
Name: child_alone, dtype: int64
0    24511
1     1669
Name: water, dtype: int64
0    23263
1     2917
Name: food, dtype: int64
0    23872
1     2308
Name: shelter, dtype: int64
0    25776
1      404
Name: clothing, dtype: int64
0    25577
1      603
Name: money, dtype: int64
0    25882
1      298
Name: missing_people, dtype: int64
0    25306
1      874
Name: refugees, dtype: int64
0    24988
1     1192
Name: death, dtype: int64
0    22739
1     3441
Name: other_aid, dtype: int64
0    24475
1     1705
Name:

In [110]:
cv_results3 = cross_validate(pipeline, X, Y, cv=3, scoring=('f1_samples'))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

In [111]:
cv_results3

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 65.5671289 ,  55.84412789,  56.3019073 ]),
 'score_time': array([ 5.35025096,  6.01594067,  6.80977654]),
 'test_score': array([ 0.34850601,  0.50005417,  0.48898858]),
 'train_score': array([ 0.78385294,  0.70733705,  0.73485949])}

In [114]:
cv_results4 = cross_validate(pipeline, X, Y, cv=3)

In [115]:
cv_results4

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 65.675421  ,  54.80811524,  55.64870548]),
 'score_time': array([ 5.22247553,  6.03239512,  6.69437766]),
 'test_score': array([ 0.26515412,  0.21782972,  0.16743067]),
 'train_score': array([ 0.79911763,  0.8278233 ,  0.82703105])}

In [117]:
cv_results5 = cross_validate(pipeline, X, Y, cv=3, scoring=('f1_weighted'))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

In [118]:
cv_results5

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 64.93734765,  56.23888016,  56.19602537]),
 'score_time': array([ 5.31334949,  6.07269669,  6.73505902]),
 'test_score': array([ 0.37522138,  0.4534976 ,  0.40630495]),
 'train_score': array([ 0.95405755,  0.95792917,  0.96177909])}

In [120]:
f1_score(Y_test, Y_pred2, average='macro') 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.21513822820998288

In [121]:
f1_score(Y_test, Y_pred2, average='micro') 

0.58444546709618039

In [122]:
f1_score(Y_test, Y_pred2) 

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.

In [123]:
f1_score(Y_test, Y_pred2, average='weighted') 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.51304767444845789

In [83]:
pipeline2 = Pipeline([
    ('vect',  CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf2', MultiOutputClassifier(KNeighborsClassifier()))
])


In [84]:
pipeline2.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [85]:
Y_pred_2=pipeline2.predict(X_test)

In [86]:
eva2=eva_fun(Y_test, Y_pred_2)

total_accuracy 0.928261607673
avg_f1_score 0.26


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 36, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [87]:
pipeline3 = Pipeline([
    ('vect',  CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf3', MultiOutputClassifier(DecisionTreeClassifier()))
])

In [88]:
pipeline3.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [89]:
Y_pred_3=pipeline3.predict(X_test)

In [90]:
eva3=eva_fun(Y_test, Y_pred_3)

total_accuracy 0.931381037263
avg_f1_score 0.6


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 36, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [115]:
### take too long time
pipeline4 = Pipeline([
    ('vect',  CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf4', MultiOutputClassifier(MLPClassifier()))
])

In [ ]:
pipeline4.fit(X_train, Y_train)

### Implement the TextLengthExtractor class

In [53]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return len(tokenize(X))

In [54]:
a='i am superman haha'

In [55]:
length=TextLengthExtractor()

In [56]:
length.transform(a)

4

### Implement the StartingVerbExtractor class

In [57]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)
    

In [58]:
b='get you haha haha'

In [59]:
verb=StartingVerbExtractor()

In [60]:
verb.transform(b)

,0
0,True


### Feature Union

In [51]:
pipeline5 = Pipeline([
    ('features', FeatureUnion([
    
        ('nlp_pipeline', Pipeline([
            ('vect',  CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('text_length', TextLengthExtractor())
        
    ])),
        
    ('mclf5', MultiOutputClassifier(RandomForestClassifier()))
])

In [52]:
pipeline5.fit(X_train, Y_train)

TypeError: expected string or bytes-like object

In [37]:
pipeline6 = Pipeline([
    ('features', FeatureUnion([
    
        ('nlp_pipeline', Pipeline([
            ('vect',  CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor())
        
    ])),
    
    ('mclf6', MultiOutputClassifier(RandomForestClassifier()))
])

In [38]:
pipeline6.fit(X_train, Y_train)

TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa1eef11b70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mclf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None

In [22]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'mclf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'mclf__estimator__bootstrap', 'mclf__estimator__class_weight', 'mclf__estimator__criterion', 'mclf__estimator__max_depth', 'mclf__estimator__max_features', 'mclf__estimator__max_leaf_nodes', 'mclf__estimator__min_impurity_decrease', 'mclf__estimator__min_impurity_split', 'mclf__estimator__min_samples_leaf', 'mclf__estimator__min_samples_split', 'mclf__estimator__min_weight_fraction_leaf', 'mclf__estimator__n_estimators', 'mclf__estimator__n_jobs', 'mclf__estimator__oob_score', 'mclf__estimator__random_state', 'mclf__estimator__verbose', 

In [23]:
parameters = {
    'mclf__estimator__n_estimators': [10, 50],
    'mclf__estimator__min_samples_split': [2, 3]
}

cv = GridSearchCV(pipeline, param_grid=parameters)


In [24]:
cv.fit(X_train, Y_train)
Y_pred3=cv.predict(X_test)

In [25]:
classreport(Y_test, Y_pred3)

related
             precision    recall  f1-score   support

          0       0.74      0.31      0.44      1477
          1       0.82      0.97      0.89      5019

avg / total       0.80      0.81      0.78      6545

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5445
          1       0.89      0.42      0.57      1100

avg / total       0.89      0.89      0.88      6545

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6520
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6545

aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3773
          1       0.79      0.61      0.69      2772

avg / total       0.77      0.77      0.76      6545

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


In [26]:
re2=classreport(Y_test, Y_pred3)

related
             precision    recall  f1-score   support

          0       0.74      0.31      0.44      1477
          1       0.82      0.97      0.89      5019

avg / total       0.80      0.81      0.78      6545

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5445
          1       0.89      0.42      0.57      1100

avg / total       0.89      0.89      0.88      6545

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6520
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6545

aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3773
          1       0.79      0.61      0.69      2772

avg / total       0.77      0.77      0.76      6545

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


AttributeError: 'NoneType' object has no attribute 'dtype'

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
pklfilename='picklefile.pkl'
gridcvpkl=open(pklfilename,'wb')
pickle.dump(cv, gridcvpkl)
gridcvpkl.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [15]:
engine2 = create_engine('sqlite:///Cleandata.db')
df2 = pd.read_sql("SELECT * FROM Cleandata", engine2)

In [16]:
df2.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
engine3 = create_engine('sqlite:///Cleandata.db')
df3 = pd.read_sql_table('Cleandata', engine3)

In [18]:
df3.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
genre_counts = df3.groupby('genre').count()['message']

In [20]:
genre_counts

genre
direct    10747
news      13039
social     2394
Name: message, dtype: int64

In [27]:
categoryname=list(df3.columns)[4:]

In [28]:
categoryname

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [30]:
len(categoryname)

36

In [29]:
totalnum=[]
for cname in categoryname:
    totalnum.append(sum(df3[cname]==1))

In [32]:
totalnum

[20064,
 4464,
 118,
 10841,
 2081,
 1311,
 724,
 471,
 859,
 0,
 1669,
 2917,
 2308,
 404,
 603,
 298,
 874,
 1192,
 3441,
 1705,
 1199,
 1331,
 532,
 159,
 283,
 120,
 309,
 1151,
 7286,
 2149,
 2440,
 282,
 2452,
 528,
 1376,
 5064]

In [31]:
len(totalnum)

36

In [24]:
sum(df3.related==1)

20064

In [21]:
df3[df3.related==1].count()

id                        20064
message                   20064
original                   6763
genre                     20064
related                   20064
request                   20064
offer                     20064
aid_related               20064
medical_help              20064
medical_products          20064
search_and_rescue         20064
security                  20064
military                  20064
child_alone               20064
water                     20064
food                      20064
shelter                   20064
clothing                  20064
money                     20064
missing_people            20064
refugees                  20064
death                     20064
other_aid                 20064
infrastructure_related    20064
transport                 20064
buildings                 20064
electricity               20064
tools                     20064
hospitals                 20064
shops                     20064
aid_centers               20064
other_in